# Processing Fault Maps

In [1]:
import geopandas as gpd


faults = gpd.read_file("../01_processed_data/gem_active_faults.geojson")
faults

,accuracy,activity_confidence,average_dip,average_rake,catalog_id,catalog_name,dip_dir,downthrown_side_dir,downthrown_side_id,epistemic_quality,...,name,net_slip_rate,notes,reference,shortening_rate,slip_type,strike_slip_rate,upper_seis_depth,vert_sep_rate,geometry
0,None,None,"(38,,)","(90.0,,)",UCF_2,UCERF3,E,None,None,None,...,Mount Diablo Thrust,"(1.55,0.8,2.22)",None,None,None,Reverse,None,"(8.0,,)",None,"LINESTRING (-121.8229 37.7301, -122.0388 37.8771)"
1,None,None,"(90,,)","(180.0,,)",UCF_9,UCERF3,None,None,None,None,...,Ortigalita (South),"(1.23,0.35,2.75)",None,None,None,Dextral,None,"(0.0,,)",None,"LINESTRING (-121.1429 37.1081, -121.138 37.034..."
2,None,None,"(90,,)","(150.0,,)",UCF_11,UCERF3,None,None,None,None,...,Monterey Bay-Tularcitos,"(0.57,0.23,1.73)",None,None,None,Dextral,None,"(0.0,,)",None,"LINESTRING (-121.50309 36.35759, -121.59573 36..."
3,None,None,"(90,,)","(180.0,,)",UCF_13,UCERF3,None,None,None,None,...,Mendocino,"(8.95,4.44,12.97)",None,None,None,Dextral,None,"(0.0,,)",None,"LINESTRING (-124.34336 40.20667, -124.41191 40..."
4,None,None,"(45,,)","(90.0,,)",UCF_15,UCERF3,NE,None,None,None,...,Table Bluff,"(0.82,0.51,1.42)",None,None,None,Reverse,None,"(0.0,,)",None,"LINESTRING (-124.17977 40.64778, -124.282 40.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16190,None,None,"(53,40,65)",None,MW_12,SMSSD,NE,None,None,None,...,Panga-4,"(NA,NA,NA)","Part of larger Panga Fault System, a Karoo age...",Castaing (1991); Habgood (1963); Chisenga et a...,None,Normal,None,None,None,"LINESTRING (34.62431 -16.24977, 34.82917 -16.4..."
16191,None,None,"(53,40,65)",None,MW_44,SMSSD,E,None,None,None,...,Tsikulamowa,"(NA,NA,NA)","Fault does not follow foliation, relatively pr...",Walshaw (1965); Bloomfield and Garson (1965),None,Normal,None,None,None,"LINESTRING (34.82039 -14.92354, 34.80048 -15.0..."
16192,None,None,"(53,40,65)",None,MW_1,SMSSD,E,None,None,None,...,Nsanje,"(NA,NA,NA)",Northern extent of Urema Graben.,Bloomfield (1957),None,Normal,None,None,None,"LINESTRING (35.24663 -16.8852, 35.13537 -17.16..."
16193,None,None,"(53,40,65)",None,MW_58,SMSSD,NE,None,None,None,...,Bilila-Mtakataka,"(0.13,0.006,0.8)",Laterally continuous steep scarp suggests rece...,Walshaw (1965); Dawson and Kirkpatrick (1968);...,None,Normal,None,None,None,"LINESTRING (34.5062 -14.26753, 34.57573 -14.43..."


In [6]:
# Load world countries from natural earth
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

# Filter to the Philippines
philippines = world[world["ADMIN"] == "Philippines"]

In [29]:
# Ensure CRS matches before spatial join/intersection
faults = faults.to_crs(philippines.crs)

# Filter fault lines that intersect the Philippines
faults_phil = gpd.overlay(faults, philippines, how='intersection')

faults_columns = faults.columns

# Filter the overlay result to only include those columns
faults_phil = faults_phil[faults_columns]

# Optional: Save to a new GeoJSON
faults_phil.to_file("faults_philippines.geojson", driver="GeoJSON")

In [30]:
faults_ph_catalog = faults[faults['catalog_name']=="philippines"]
faults_eos_se_asia = faults_phil[faults_phil['catalog_name']=="EOS_SE_Asia"]

In [ ]:
faults_eos_se_asia

92     LINESTRING (294884 2028674.606, 296123.189 202...
93     LINESTRING (242254.234 1982797.767, 246577.877...
94     LINESTRING (261321.042 1972925.044, 259993.791...
95     LINESTRING (331520.676 1949404.86, 324452.108 ...
96     LINESTRING (294564.861 1883887.411, 293877.85 ...
97     LINESTRING (233659.134 1817255.118, 234350.057...
98     LINESTRING (254726.419 1809256.416, 254936.424...
99     LINESTRING (389159.03 1578689.697, 398134.703 ...
100    LINESTRING (657891.057 1268976.58, 660689.792 ...
101    LINESTRING (766651.606 1078682.148, 770498.551...
102    MULTILINESTRING ((261321.042 1972925.044, 2599...
103    MULTILINESTRING ((233659.134 1817255.118, 2343...
104    MULTILINESTRING ((254726.419 1809256.416, 2549...
105    MULTILINESTRING ((291266.61 1751431.793, 29178...
106    MULTILINESTRING ((328795.885 1698664.907, 3408...
107    LINESTRING (308307.724 1661643.205, 307542.39 ...
Name: geometry, dtype: geometry

In [33]:
import pandas as pd

faults_ph_catalog = faults_ph_catalog.to_crs("EPSG:32651")
faults_eos_se_asia = faults_eos_se_asia.to_crs("EPSG:32651")

# Step 2: Combine them into a single GeoDataFrame
faults_ph = gpd.GeoDataFrame(
    pd.concat([faults_ph_catalog, faults_eos_se_asia], ignore_index=True),
    crs="EPSG:32651"
)

# Step 4: Save to a new GeoJSON file
faults_ph.to_file("faults_philippines.geojson", driver="GeoJSON")
faults_ph

,accuracy,activity_confidence,average_dip,average_rake,catalog_id,catalog_name,dip_dir,downthrown_side_dir,downthrown_side_id,epistemic_quality,...,name,net_slip_rate,notes,reference,shortening_rate,slip_type,strike_slip_rate,upper_seis_depth,vert_sep_rate,geometry
0,None,None,None,None,PHL_0,philippines,None,None,None,None,...,Colosas Fault,None,None,None,None,Sinistral,None,None,None,"LINESTRING (783744.141 821869.776, 777440.73 8..."
1,None,None,None,None,PHL_1,philippines,None,None,None,None,...,Colosas Fault,None,None,None,None,None,None,None,None,"LINESTRING (783744.141 821869.776, 777440.73 8..."
2,None,None,None,None,PHL_2,philippines,None,None,None,None,...,Dacudao Fault,None,None,None,None,Sinistral,None,None,None,"LINESTRING (768839.979 801831.36, 768857.815 8..."
3,None,None,"(35,,)",None,PHL_3,philippines,W,E,None,None,...,Central Cebu Fault,"(2.0, 1.0, 3.0)",None,None,None,Reverse,None,None,None,"LINESTRING (604367.967 1163271.115, 604094.051..."
4,None,None,"(35,,)",None,PHL_4,philippines,W,E,None,None,...,East Negros Fault North,"(2.0, 1.0, 3.0)",None,None,None,Reverse,None,None,None,"LINESTRING (554279.001 1185514.999, 554239.088..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,None,None,"(90.0,,)","(180.0,,)",EOS_AF1785,EOS_SE_Asia,None,None,None,None,...,None,None,None,None,None,Dextral,None,"(1.0,,)",None,"MULTILINESTRING ((233659.134 1817255.118, 2343..."
128,None,None,"(90.0,,)","(180.0,,)",EOS_AF1786,EOS_SE_Asia,None,None,None,None,...,None,None,None,None,None,Dextral,None,"(1.0,,)",None,"MULTILINESTRING ((254726.419 1809256.416, 2549..."
129,None,None,"(90.0,,)","(180.0,,)",EOS_AF1787,EOS_SE_Asia,None,None,None,None,...,None,None,None,None,None,Dextral,None,"(1.0,,)",None,"MULTILINESTRING ((291266.61 1751431.793, 29178..."
130,None,None,"(90.0,,)","(180.0,,)",EOS_AF1788,EOS_SE_Asia,None,None,None,None,...,None,None,None,None,None,Dextral,None,"(1.0,,)",None,"MULTILINESTRING ((328795.885 1698664.907, 3408..."


In [38]:
faults_ph_geom = faults_ph.geometry
faults_ph_geom

faults_ph_geom.to_file("faults_ph_geometry.geojson", driver="GeoJSON")

In [37]:
# Sample for inference
from shapely.geometry import Point

coordinates = [
    (13.4125, 122.5623),
    (10.7103, 122.5621),
    (16.4023, 120.596),
    (15.489, 120.9739),
    (14.5995, 120.9842),
    (8.051, 124.92),
    (6.9214, 122.079),
    (11.0046, 124.6093),
    (13.54, 122.5623),
    (17.6133, 121.7269),
]


points_df = pd.DataFrame(coordinates, columns=["lat", "lon"])
points_geometry = [Point(lon, lat) for lat, lon in coordinates]  # Note: (lon, lat) for Point
points = gpd.GeoDataFrame(points_df, geometry=points_geometry, crs="EPSG:4326")

# --- Step 4: Reproject both to UTM or a metric CRS for accurate distance (Philippines: UTM zone 51N = EPSG:32651) ---
faults_proj = faults_ph_geom.to_crs(epsg=32651)
points_proj = points.to_crs(epsg=32651)

# --- Step 5: Compute minimum distance from each point to the nearest fault line ---
points_proj["distance_to_fault_m"] = points_proj.geometry.apply(
    lambda point: faults_proj.distance(point).min()
)

# --- Step 6: Output results ---
results = points_proj[["lat", "lon", "distance_to_fault_m"]]
print(results)

       lat       lon  distance_to_fault_m
0  13.4125  122.5623          8791.423406
1  10.7103  122.5621          8981.293215
2  16.4023  120.5960          9149.599699
3  15.4890  120.9739         15833.282125
4  14.5995  120.9842          9433.453397
5   8.0510  124.9200         19418.723746
6   6.9214  122.0790         12838.622612
7  11.0046  124.6093         10613.225991
8  13.5400  122.5623          8552.087192
9  17.6133  121.7269         25731.144104
